In [32]:
import os
import pandas as pd
import numpy as np

In [5]:
def listdir(path):
    ls = os.listdir(path)
    return [os.path.join(path, f) for f in ls]

In [36]:
dfs = list(map(pd.read_csv, listdir('./bench/suite-10-backtracking/')))

In [37]:
data = pd.concat(dfs)
data.drop('Unnamed: 0', inplace=True, axis=1)
data.head()

,seed,time,solved,visited,msg,seq,seqlen,datetime,shootme
0,1432,38.208051,False,200001,exceeded max states,NaN,-1,2019-11-26T21:15:40.303174,Solved-Min
1,2337,53.123784,False,200001,exceeded max states,NaN,-1,2019-11-26T21:15:55.219274,Solved-Min
2,2919,0.031976,True,122,NaN,12 DR8 W6 W1 DR7 W1 W7 71-2 DR7 DR6 W7 DR8 DR6...,427,2019-11-26T21:15:02.127650,Solved-Min
3,7718,0.039170,True,141,NaN,75 73 52-2 51 37-2 35 26-3 42 74-3 76 DR8 DR7 ...,488,2019-11-26T21:15:02.134715,Solved-Min
4,4691,0.038805,True,152,NaN,7D 25 2C 13 DR8 DR7 DR7 DR7 DR7 DR7 DR7 W7 W3 ...,430,2019-11-26T21:15:02.134812,Solved-Min


In [38]:
solved = data[data.solved == True]
print(f"{len(solved)} solved out of {len(data)} = {np.average(data.solved)*100:.1f}%")

58 solved out of 100 = 58.0%
